In [52]:
import pandas as pd
import numpy as np
from estimator_model.estimation_learner import meta_learner
from estimator_model import ml_model

It's still not clear why XLearner and TLearner are not effective.

In [54]:
n_users = 10000
def gen_data(n_users, binary_treatment=True, with_income=False):
    
    if with_income:
        income = np.random.normal(500, scale=15, size=n_users)
        gender = np.random.randint(0, 2, size=n_users)
        coupon = gender * 20 + 110 + income / 50 + np.random.normal(scale=5, size=n_users)
        if binary_treatment:
            coupon = (coupon > 120).astype(int)
        amount = coupon * 150 + gender * 100 + 150 + income / 5 + np.random.normal(size=n_users)
        time_spent = coupon * 10 + amount / 10

        df = pd.DataFrame({
            'gender': gender,
            'coupon': coupon,
            'amount': amount,
            'income': income,
            'time_spent': time_spent,
        })

        return df
    else:
        gender = np.random.randint(0, 2, size=n_users)
        coupon = gender * 20 + 150 + np.random.normal(scale=5, size=n_users)
        if binary_treatment:
            coupon = (coupon > 150).astype(int)
        amount = coupon * 30 + gender * 100 + 150 + np.random.normal(size=n_users)
        time_spent = coupon * 100 + amount / 10

        df = pd.DataFrame({
            'gender': gender,
            'coupon': coupon,
            'amount': amount,
            'time_spent': time_spent,
        })
        
        return df

df = gen_data(n_users, with_income=True)
df['treatment'] = df['coupon'] > 0
df

,gender,coupon,amount,income,time_spent,treatment
0,0,1,392.615139,465.212789,49.261514,True
1,0,0,248.698072,485.902416,24.869807,False
2,0,1,402.419307,511.520948,50.241931,True
3,1,1,499.715426,498.731233,59.971543,True
4,1,1,497.594554,486.285776,59.759455,True
...,...,...,...,...,...,...
9995,1,1,501.816475,511.725780,60.181648,True
9996,1,1,497.091514,485.233299,59.709151,True
9997,1,1,500.184297,507.400975,60.018430,True
9998,1,1,497.412782,484.362127,59.741278,True


In [64]:
df['amount'].value_counts().shape[0]

10000

In [42]:
a = np.array([1 for i in df['income']])

In [47]:
len(np.ones_like(df['income']))

10000

In [51]:
df.index

RangeIndex(start=0, stop=10000, step=1)

In [41]:
(df['income'] / df['income']).mean()

1.0

In [20]:
new_df[new_df['time_spent'] > 26]['coupon'].value_counts()

1    7505
0       2
Name: coupon, dtype: int64

## SLearner

In [21]:
ml = ml_model.LR()
l = meta_learner.SLearner(ml_model=ml)

In [22]:
l.prepare(data=new_df, outcome='amount', treatment='coupon', adjustment={'gender', 'income'})

array([150.02196995, 150.02196995, 150.02196995, ..., 150.02196995,
       150.02196995, 150.02196995])

In [23]:
l.estimate_ate(data=new_df, 
               outcome='amount', 
               treatment='coupon', 
               adjustment=['gender', 'income'])

150.02196994838965

In [24]:
l.estimate_cate(data=new_df, 
                outcome='amount', 
                treatment='coupon', 
                adjustment=['gender', 'income'],
                condition_set={'time_spent'},
                condition=(new_df['time_spent'] >= 25.06))

149.71783769915066

## TLearner

In [25]:
t = meta_learner.TLearner(ml_model=ml)
t.estimate_ate(data=new_df, 
               outcome='amount', 
               treatment='coupon', 
               adjustment=['gender', 'income'])

199.63088066696852

In [26]:
t.estimate_cate(data=new_df, 
                outcome='amount', 
                treatment='coupon', 
                adjustment=['gender', 'income'],
                condition_set={'time_spent'},
                condition=(new_df['time_spent'] >= 20))

199.63088066696852

## XLearner

In [27]:
x = meta_learner.XLearner(ml_model=ml)
x.estimate_ate(data=new_df, 
               outcome='amount', 
               treatment='coupon', 
               adjustment=['gender', 'income'])

24.804423189303208

$\bot$

In [28]:
a = '-'
for i in range(9):
    a += '-'
a

'----------'